In [1]:
import os, tqdm, time, random
import cv2 as cv
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import models, layers, losses, optimizers

from loss import *
from utils import *
from network import *

random.seed(777)
tf.set_random_seed(777)
np.random.seed(777)

In [2]:
root_dir = '../data/for_section/'

fold_idx = 1

train_path = os.path.join(root_dir, 'for_fold_%d'%fold_idx)

val_path = os.path.join(root_dir, 'for_fold_%d_val'%fold_idx)

In [3]:
test = data_loader(train_path)

In [4]:
data = []
label = []
n_size = 32
n_slice = 16
cs_strides = 16
a_strides = 2
cnt = 0
cor, sag, axi = test.shape
#print(test.shape)
for idx in tqdm.tqdm_notebook(range(0, cor-n_size, cs_strides)):
    for jdx in range(0, sag-n_size, cs_strides):
        for kdx in range(0, axi-(n_slice*6), a_strides):
            tmp = test[idx:idx+n_size, jdx:jdx+n_size, kdx:kdx+(n_slice*6)]
            if tmp.mean() > 225:
                label.append(tmp)
                tmp = np.array(np.dsplit(tmp, n_slice))
                tmp = tmp.mean(axis=-1)
                tmp = np.transpose(tmp, [1,2,0])
                data.append(tmp)
                
label = np.array(label)[..., np.newaxis]
data = np.array(data)[..., np.newaxis]
print(data.shape)
print(label.shape)


(3317, 32, 32, 16, 1)
(3317, 32, 32, 96, 1)


In [5]:
net = SR3D()

W0917 12:32:34.854861 140330720319232 deprecation.py:506] From /root/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
date = time.ctime().split(' ')

ckpt_root = './checkpoint/%s_%02d_%s/DeepSR_msemi_test'%(date[1], int(date[2]), date[-1])
result_root = './result/%s_%02d_%s/DeepSR_msemi_test'%(date[1], int(date[2]), date[-1])

try:
    os.makedirs(ckpt_root)
    print("\nMake Save Directory!\n")
except:
    print("\nDirectory Already Exist!\n")

try:
    os.makedirs(result_root)
    print("\nMake Save Directory!\n")
except:
    print("\nDirectory Already Exist!\n")
    

model_json = net.to_json()
with open(os.path.join(ckpt_root, "model.json"), "w") as json_file:
    json_file.write(model_json)
print("\nModel Saved!\n")


Make Save Directory!


Make Save Directory!


Model Saved!



In [7]:
net.compile(optimizer=optimizers.Adam(0.0001), loss=mse_mi, metrics=['mse', gradient_3d_loss, mutual_information])

W0917 12:32:38.085538 140330720319232 deprecation.py:323] From /root/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
history = net.fit(data, label, batch_size=16, epochs=100)


net.save_weights(os.path.join(ckpt_root, 'weight.h5'))

df = pd.DataFrame(history.history)
df.to_csv(os.path.join(result_root, 'loss.csv'))

Epoch 1/100
3317/3317 [==============================] - 848s 256ms/sample - loss: 9416.0256 - mean_squared_error: 11551.2559 - gradient_3d_loss: 397791.7188 - mutual_information: -2.1354
Epoch 2/100
3317/3317 [==============================] - 837s 252ms/sample - loss: 3794.3590 - mean_squared_error: 5939.2129 - gradient_3d_loss: 288384.7188 - mutual_information: -2.1449
Epoch 3/100
3317/3317 [==============================] - 845s 255ms/sample - loss: 3358.5377 - mean_squared_error: 5510.7329 - gradient_3d_loss: 272810.9688 - mutual_information: -2.1523
Epoch 4/100
3317/3317 [==============================] - 843s 254ms/sample - loss: 3079.1030 - mean_squared_error: 5233.3286 - gradient_3d_loss: 262454.1250 - mutual_information: -2.1542
Epoch 5/100
3317/3317 [==============================] - 843s 254ms/sample - loss: 2861.2703 - mean_squared_error: 5017.4307 - gradient_3d_loss: 254644.6094 - mutual_information: -2.1563
Epoch 6/100
3317/3317 [==============================] - 842s 25

In [9]:
scan_list = sorted(os.listdir(val_path))[1:]
for scan in scan_list:
    dante_path = os.path.join(val_path, scan, 'T1SPACE09mmISOPOSTwDANTE')
    img_name = [i for i in os.listdir(dante_path) if '.nii' in i and '_rsl' not in i][0]
    #print(img_name)
    val = nib.load(os.path.join(dante_path, img_name))
    val = check_data(val.get_data())
    
val_in = []
cor, sag, axi = val.shape
tmp = np.array(np.dsplit(val, axi//6))
tmp = tmp.mean(axis=-1)
tmp = np.transpose(tmp, [1, 2, 0])

half_top = int(np.floor(tmp.shape[-1]/2))
#half_bot = int(np.ceil(tmp.shape[-1]/2))

In [10]:
recon = np.zeros(shape=[cor, sag, axi])

test = {}
sli = 12
for i in range(0, tmp.shape[-1], sli):
    print(i)
    test['test_3d_%d'%i] = net.predict(tmp[np.newaxis, ..., i:i+sli, np.newaxis])
    recon[..., i*6:(i+sli)*6] = test['test_3d_%d'%i][0, ..., 0]
# test_3d_2 = net.predict(tmp[np.newaxis, ..., half_top:, np.newaxis])

0
12


In [11]:
aff = np.eye(4)
aff[2, 2]=6

nib.save(nib.Nifti1Image(tmp, aff), os.path.join(result_root, 'val_input.nii'))
nib.save(nib.Nifti1Image(recon, np.eye(4)), os.path.join(result_root, 'val_pred.nii'))
nib.save(nib.Nifti1Image(val, np.eye(4)), os.path.join(result_root, 'val_label.nii'))